In [1]:
import os
from collections import defaultdict
import copy
from functools import partial

## Парсинг папки и оценивание

In [2]:
class ContestsEvaluator():

    ALL = {'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X'}
    
    def __init__(self, my_students: dict, 
                 submit_folder: str, 
                 last_submit_on_time: int,
                 extra_tasks: list = [],
                 extra_point_threshold : int = 1):
        
        self.STUDENTS = my_students
        self.submit_folder = submit_folder
        self.extra_tasks = set(extra_tasks)
        self.last_before_deadline = int(last_submit_on_time)
        
        self.all_folders = [st_folder for st_folder in os.listdir(submit_folder) 
                            if os.path.isdir(os.path.join(submit_folder, st_folder))]
        self.compulsory = self.ALL - self.extra_tasks
        self.max = len(self.compulsory)
        self.extra_point_threshold = extra_point_threshold
        
        template = {'ok_submits': {},
                   'points': 0,
                   'extra_point': 0,
                   'after_deadline_tasks': {}}
        
        self.students_results = {self.STUDENTS[st]: copy.deepcopy(template) for st in self.STUDENTS}
        
    def _find_my_students(self):
        """Заполняет словарь вида {фио студента: путь к папке с его посылками}"""
        
        self.my_students_folders = {}
        for folder in self.all_folders:
            student = '-'.join(folder.split('-')[:-1])
            if student in self.STUDENTS.values():  # если название папки содежит фио
                self.my_students_folders[student] = os.path.join(self.submit_folder, folder)
            elif student in self.STUDENTS:  # если название содержит логин студента
                self.my_students_folders[STUDENTS[student]] = os.path.join(self.submit_folder, folder)
            elif len([st for st in self.STUDENTS.values() if st.startswith(student)]):  # если в названии фио не полностью
                student = [st for st in self.STUDENTS.values() if st.startswith(student)][0]
                self.my_students_folders[student] = os.path.join(self.submit_folder, folder)
    
    def _ok_solutions(self, solutions_list):
        """Возвращает словарь вида {задача: [посылки со статусом ok]}"""
        
        ok = defaultdict(list)
        for solution in solutions_list:
            properties = solution.split('-')
            status = properties[-1]
            task = properties[0]
            send_id = properties[1]
            if status == 'OK.py':
                ok[task].append(send_id)
        return ok
    
    def _extra_point(self, done_extra):
        """Засчитывает доп балл, если количество зачтенных доп задач больше какого-то порога"""
        
        extra_point = 0
        if len(done_extra) >= self.extra_point_threshold:
            extra_point += 1
        return extra_point
    
    def _student_ok_submits(self, student):
        """Cохраняет для студента список посылок с кодом ok по каждой задаче"""
        
        student_folder = self.my_students_folders[student]
        all_submits = os.listdir(student_folder)
        right_solutions = self._ok_solutions(all_submits)
        self.students_results[student]['ok_submits'] = right_solutions
        
    def count_points(self, extra_point_rule = None):
        """Считает баллы для всех студентов. 
        За каждую задачу, по которой есть посылка с кодом ok, - 1 балл. 
        Если все правильные посылки по задаче после дедлайна, то за эту задачу 0 баллов.
        Доп баллы считаются по правилу extra_point_rule, которое получает на вход аргумент done_extra - количество сделанных
        доп задач. По дефолту правило такое: если сделанных задач больше self.extra_point_threshold, то балл дается."""
        
        for student in self.my_students_folders:
            
            right_tasks = self.students_results[student]['ok_submits'].keys() 
            
            self._after_deadline_tasks(student)
            ignore_tasks = self.students_results[student]['after_deadline_tasks']
            right_tasks = set(right_tasks) - set(ignore_tasks)
            
            points = len(right_tasks & self.compulsory)
            
            done_extra = right_tasks & self.extra_tasks
            if not extra_point_rule:
                extra_point = self._extra_point(done_extra)
            else:
                rule = partial(extra_point_rule, done_extra=done_extra)
                extra_point = rule()
                
            all_points = {
                'points': points,
                'extra_point': extra_point
            }
            self.students_results[student].update(all_points)
    
    def parse_contest(self):
        """Достает из папки со всеми посылками только те, которые прошли все тесты и которые от студентов из списка."""
        
        self._find_my_students()
        for student in self.my_students_folders:
            self._student_ok_submits(student)
            
    def _after_deadline_tasks(self, student):
        """Ищет задачи, все ok посылки по которым, студент отправил после дедлайна"""
        
        submits = self.students_results[student]['ok_submits']
        after_deadl_submits = [submit for task in submits for submit in submits[task] if int(submit) > self.last_before_deadline]
        
        tasks_to_ignore = []
        for task in submits:
            if not (set(submits[task]) - set(after_deadl_submits)):
                tasks_to_ignore.append(task)
        self.students_results[student]['after_deadline_tasks'] = tasks_to_ignore
        
    def get_results(self):
        """Возвращает информацию по каждому студенту: 
            фио
            сделанные обязательные задачи (без учета дедлайна)
            сделанные доп задачи (без учета дедлайна)
            баллы (с учетом дедлайна)
            доп баллы (с учетом дедлайна)
            задачи, сданные после дедлайна"""
        
        results = []
        for student in sorted(self.students_results):
            submitted = sorted(self.students_results[student]['ok_submits'].keys())
            compulsory = ' '.join([task for task in submitted if task in self.compulsory]) # без учета дедлайна
            extra = ' '.join([task for task in submitted if task in self.extra_tasks]) # без учета дедлайна
            points = self.students_results[student]['points']
            extra_points = self.students_results[student]['extra_point']
            after_deadline = ' '.join(self.students_results[student]['after_deadline_tasks'])
            results.append([student,
                            compulsory, 
                            extra, 
                            points, 
                            extra_points, 
                            after_deadline])
        return results
    
    def get_hw_parameters(self):
        """Возвращает информацию по домашке: список обязательных задач, список доп задач, максимальный балл"""
        
        parameters = [
            ['compulsory tasks', ' '.join(sorted(self.compulsory))],
            ['extra tasks', ' '.join(sorted(self.extra_tasks))],
            ['max', self.max]
        ]
        return parameters

In [3]:
# {логин: фио} (потому что иногда в посылках не фио, а логин)
STUDENTS = { 
    'hse-minor-2022-5': "Агаджанов Хусейин Рафиг оглы", 
    "hse-minor-2022-20": "Баталова Анна Алексеевна", 
    "hse-minor-2022-32": "Бромирская Анна Сергеевна",
    "hse-minor-2022-60": "Гришин Николай Константинович",
    "hse-minor-2022-69": "Дмитриев Александр Антонович",
    "hse-minor-2022-95": "Каневский Дмитрий Маркович",
    "hse-minor-2022-141": "Максюта Софья Сергеевна",
    "hse-minor-2022-153": "Михайлова Элина Эдуардовна",
    "hse-minor-2022-273": "Негробова Полина Дмитриевна",
    "hse-minor-2022-175": "Одинцов Олег Александрович",
    "hse-minor-2022-195": "Рассудимова Надежда Михайловна",
    "hse-minor-2022-216": "Тарбеев Вячеслав Владимирович",
    "hse-minor-2022-227": "Федорова Екатерина Михайловна",
    "hse-minor-2022-240": "Цыкунов Никита Дмитриевич",
    "hse-minor-2022-260": "Щеголев Даниил Витальевич",
}

path_to_folder = '.\submits_2' #путь к папке с распакованным архивом всех посылок
hw = 'дз2'
extra_tasks = ['A', 'T', 'U', 'W', 'X'] # задачи со звездочкой

# NB! на странице со всеми посылками время +3 часа (последняя посылка будет следующим днем до 02:59:59)
last_on_time = '71649855'

extra_point_threshold = 1 # сколько задач со звездочкой надо решить, чтобы получить доп балл

In [4]:
cont_eval = ContestsEvaluator(my_students=STUDENTS,
                              submit_folder=path_to_folder, 
                              last_submit_on_time = last_on_time,
                              extra_tasks=extra_tasks,
                              extra_point_threshold=extra_point_threshold)
cont_eval.parse_contest() 
cont_eval.count_points()

## Запись результатов в файл

### Запись в гугл табличку

In [6]:
import gspread

In [7]:
gc = gspread.service_account(filename='credentials.json')
sh = gc.open("iad-hw-checking")
hw_sheet = sh.worksheet(hw)

In [8]:
hw_sheet.update('A19:B21', cont_eval.get_hw_parameters())
hw_sheet.update('A2:F16', cont_eval.get_results())

{'spreadsheetId': '1YSLCzG70fxVhJHBgqMCHkHvEonaIP1bmpvHZbKHvjms',
 'updatedRange': "'дз2'!A2:F16",
 'updatedRows': 15,
 'updatedColumns': 6,
 'updatedCells': 90}

### Запись в csv файл

In [9]:
import csv

In [23]:
path_to_csv = os.path.join(path_to_folder, f'{hw}.csv')  # путь к файлу с результами
 
with open(path_to_csv, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile, delimiter=',',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    writer.writerow(['Student', 'Done compulsory', 'Done extra', 'Points', 'Extra point', 'After deadline tasks'])
    writer.writerows(cont_eval.get_results())
    writer.writerows([[], []])
    writer.writerows(cont_eval.get_hw_parameters())
    
os.path.abspath(path_to_csv)

## Создание файлов с решениями

In [12]:
import shutil

In [13]:
def aggregate_submits(evaluated_contest):
    """Собирает по задачам все посылки (со статусом OK и до дедлайна) от студентов, которых оцениванием."""
    
    task_submits = defaultdict(list) # {task: [(submit_path, student_name)]}
    for student in evaluated_contest.my_students_folders:
        folder = evaluated_contest.my_students_folders[student]
        for file in os.listdir(folder):
            properties = file.split('-')
            status = properties[-1]
            task = properties[0]
            submit_id = properties[1] 
            if status == 'OK.py' and int(submit_id) <= evaluated_contest.last_before_deadline:
                task_submits[task].append((os.path.join(folder, file), student))  
    return task_submits

In [20]:
def write_aggregated(evaluated_contest_obj,
                     folder_path = os.path.join(path_to_folder, '_my-students-submits')):
    """Создает файлы для каждой задачи. Каждый файл содержит все учитываемые посылки по конкретной задаче.
    Каждая посылка отделена от другой пустой строкой. Перед каждым решением в коментариях указано чье решение."""
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    
    task_submits = aggregate_submits(evaluated_contest_obj)
    for task in task_submits:
        task_solutions = []
        for submit_path, student in task_submits[task]:
            with open(submit_path, 'r', encoding='utf-8') as f:
                    code = f.read()
            task_solutions.append('#' + student + '\n' + code + '\n\n')

        newf_name = os.path.join(folder_path, f'task_{task}.py')
        with open(newf_name, 'w', encoding='utf-8') as newf:
            newf.writelines(task_solutions)
            
    return os.path.abspath(folder_path)

In [21]:
# путь к папке, где будут храниться решения, агресгированные по задачам
folder_path = os.path.join(path_to_folder, '_my-students-submits') 

write_aggregated(cont_eval, folder_path)